In [1]:
%reload_ext watermark
%watermark -a 'Antonio Javier González Ferrer & Hamed Mohammadpour' -v -d -r

# Introduction

In this notebook, we will study the problem of discovering association rules between itemsets in a sales transaction database. There are two principal subproblems to solve:

1. Finding frequent itemsets with support at least $s$.
2. Generating association rules with confidence at least $c$ from the itemsets found in the first step.

Therefore, the ultimate goal is to find association rules $X \rightarrow Y$, where $X$ and $Y$ are itemsets such that $X \cap Y = \emptyset$.

In [2]:
import logging
    
from apriori.utility import create_items_catalog, printify
from apriori.itemset import Itemset
from apriori.association_rule import AssociationRule

# Frequent Itemsets

Let us explain how the frequent itemsets are calculated with a little example. In this case, our shopping list consists of three different baskets, where each of them contains different elements. We will use letters for representing the elements and numbers for representing baskets. 

In [3]:
baskets = [['a', 'b', 'c'], ['a', 'b', 'e'], ['c', 'd']]
n = len(baskets)
baskets

[['a', 'b', 'c'], ['a', 'b', 'e'], ['c', 'd']]

In [4]:
baskets[0]

['a', 'b', 'c']

As we can see, the basket number $0$ contains the elements $a$, $b$ and $c$. An element $i$ is said to be "frequent" if it appears in more than a certain number of baskets. We call this count the support $s(i)$ and we measure it proportionally to the total numbers of transactions. For instance, $s(a) = 2/3$ since it appears in two out of the three baskets. If we would set a support threshold of $0.5$, then we would say that $a$ is a frequent itemset of size $1$.

We can also calculate frequent itemsets of any size $k$. Let us focus on calculating frequent itemsets of size $k=2$ for the sake of simplicity. A näive algorithm would count first the frequent itemsets of size $1$, then combinate all different pairs from the previous set and count again the occurrences of the itemsets of size $2$ within the baskets. Considering that we have to check the number of appearances for each pair (which is the most costly operation), there are two main drawbacks in terms of efficiency using this approach:

1. We do not need to iterate over the whole dataset again for calculating the support of the pairs, since we could have stored the number of occurrences of each element in a clever way in the first pass. We will explain it soon but the method `create_items_catalog()` saves for each item the list of baskets where it appears (do you already realise how to use this information for calculating the support of 2-itemsets?).

2. The <i>apriori</i> property: each subset of a frequent itemset $k$ must be also a frequent itemset. Imagine we generate the tuple $(a,b,c)$ given that the frequent itemsets of size $2$ are $\{(a,b), (b,c)\}$. The tuple $(a,b,c)$ cannot belong to the frequent itemsets of size $3$ since its subset $(a,c)$ is not a frequent itemset of size 2. Therefore, a bruce force generation of candidate frequent itemsets is not efficient. We will handle this generation with the implementation of the `generate_candidates()` method.

## Efficient Store of Support

The key idea is to pass over the dataset only once and store for each of the items a list of baskets where it appears. This implicitly indicates the support of an element just by dividing the length of its list by the total number of baskets. Using this information, we can trivially calculate the support of an element of size $k$. We just need to intersect the $k$ different lists and the length of the resulting set will be the final support.

In [5]:
items_catalog = create_items_catalog(baskets, 'str')
print(items_catalog)

defaultdict(<class 'list'>, {'a': [0, 1], 'e': [1], 'b': [0, 1], 'd': [2], 'c': [0, 2]})


We can see that the element $a$ is in the lists $0$ and $1$ and the element $c$ in the lists $0$ and $2$, then $s(a)=s(c)=2/3$. Hence, if we want to calculate the support of the itemset $(a, c)$ we just need to intersect both lists and see the length:

In [6]:
ac_intersection = set(items_catalog['a']).intersection(set(items_catalog['c']))
ac_occurrences = len(ac_intersection)
print("The intersection of (a,c) is {} and the support is {}/{}.".format(ac_intersection, ac_occurrences, n))

The intersection of (a,c) is {0} and the support is 1/3.


## Generation of Candidate Itemsets

The candidates itemsets of size $k$ are generated by combining the frequent itemsets of size $k-1$ and the singletons (frequent itemsets of size $1$). However, each candidate itemset must fulfil the <i>apriori</i> property. Let us compare the brute force method against the implemented method:


In [7]:
def generate_brute_force_candidates(L2, L1):
    brute_force_candidates = set()
    for candidate in L2:
        for single in L1:
            k_candidate = frozenset(candidate.union(single))
            if len(k_candidate) == 3:
                brute_force_candidates.add(k_candidate)
    return brute_force_candidates

In [8]:
L1 = ['a', 'b', 'c', 'd']

# After pruning itemsets, these meet the support threshold...
L2 = [set({'a', 'b'}), set({'a', 'c'}), set({'b', 'c'}), set({'b', 'd'})]

brute_force_candidates = generate_brute_force_candidates(L2, L1)
apriori_candidates = Itemset().generate_candidates(L2, L1)

In [9]:
print('The brute force candidates are: \n{}\n'.format(printify(brute_force_candidates, 4)))
print('The candidates generated by the apriori algorithm are: \n{}'.format(printify(apriori_candidates)))

The brute force candidates are: 
{(b, a, c), (b, d, c), (b, d, a), (a, d, c)}

The candidates generated by the apriori algorithm are: 
{(b, a, c)}


As expected, the only valid itemset of size $k=3$ is $(a,b,c)$ since $(a,b), (a,c)$ and $(b,c)$ are also frequent itemsets. For instance, $(a, d, b)$ contains $(a,d)$ which is not a frequent itemset.

# Real Example

Now let us test the implementation of the apriori algorithm and generation of associtiation rules with a real example. The following dataset is a sale transaction dataset and was generated from the IBM Almaden Quest research group and can be downloaded from [here](http://fimi.ua.ac.be/data/T10I4D100K.dat). 

## Load the data

In [10]:
data_file = "./data/T10I4D100K.dat"

with open(data_file, 'r') as f:
    content = f.read()
    baskets = []

    for line in content.splitlines():
        baskets.append(line.split())

print("The dataset contains {:,} baskets.".format(len(baskets)))

The dataset contains 100,000 baskets.


## Defining support threshold and Itemset class
Let us consider that an itemset is frequent is its support is bigger than 0.01 (that is it appears in, at least, 1% of the transactions):

In [11]:
support = 0.01
itemset = Itemset(baskets, support)

First, we create the items catalog from the baskets. Recall that this object will contain for each individual item a list of baskets where the item appears:

In [12]:
%%time
items_catalog = create_items_catalog(baskets)
items_length = len(list(items_catalog.keys()))

print("There are {} different items.".format(items_length))

There are 870 different items.
CPU times: user 476 ms, sys: 8 ms, total: 484 ms
Wall time: 483 ms


## Singletons (frequent itemsets k=1)
We introduce the following notation. The set $C_k$ contains the candidate itemsets of size $k$, whereas the set $L_k$ contains the actual frequent itemsets of size $k$. In the next step, we calculate which elements of size $1$ are actually frequent itemsets, i.e. those who meet the specified support:

In [13]:
%%time
frequent_itemsets = set()
c1 = {frozenset({x}) for x in set(items_catalog.keys())}
l1, _ = itemset.prune_candidates(c1, items_catalog)
print("There are {} frequent itemsets of size k=1.".format(len(l1)))

There are 375 frequent itemsets of size k=1.
CPU times: user 88 ms, sys: 0 ns, total: 88 ms
Wall time: 88.3 ms


Some of these elements are:

In [14]:
printify(l1, 10)

'{(349), (789), (653), (862), (490), (884), (151), (746), (579), (422)}'

## Generate candidate itemsets and prunning
From those singletons, let us see how many candidate itemsets of size $k=2$ we find:

In [15]:
%%time
c2 = itemset.generate_candidates(l1, l1)
print("There are {} possible candidate itemsets of size k=2.".format(len(c2)))

There are 70125 possible candidate itemsets of size k=2.
CPU times: user 324 ms, sys: 0 ns, total: 324 ms
Wall time: 323 ms


Now we have to find the support of those elements and filter out the ones that do not fulfill the threshold:

In [16]:
%%time 
l2, _ = itemset.prune_candidates(c2, items_catalog)
print("There are {} frequent itemsets of size k=2.".format(len(l2)))

There are 9 frequent itemsets of size k=2.
CPU times: user 16.4 s, sys: 16 ms, total: 16.4 s
Wall time: 16.4 s


In [17]:
printify(l2, 10)

'{(825, 704), (217, 346), (829, 368), (789, 829), (682, 368), (39, 825), (39, 704), (390, 227), (390, 722)}'

Counting candidate pairs is the operation that takes most of the time within the algorithm. The `items_catalog` idea improves the performance time, but still it needs to check the intersection of the two lists for each candidate. In the appendix, we propose an alternative method that improves the time complexity at the expense of making the space complexity worse.

In practice, for larger $k$ sets the possible combinations are much smaller. We expect to find more frequent pairs than frequent triples, more frequent triples than frequent quadruples, and so on. Therefore, the rest of the algorithm should be fast:

In [18]:
%%time
c3 = itemset.generate_candidates(l2, l1)
print("There is {} possible candidate itemset of size k=3.".format(len(c3)))

There is 1 possible candidate itemset of size k=3.
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 6.16 ms


In [19]:
%%time
l3, _ = itemset.prune_candidates(c3, items_catalog)
print("There is {} frequent itemset of size k=3.".format(len(l3)))

There is 1 frequent itemset of size k=3.
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.42 ms


In [20]:
printify(l3, 1)

'{(825, 39, 704)}'

## All-in-one: Finding k frequent itemsets with support at least s 
Finally, we can do the whole process automatically, that is, without specifying the values of $k$ by hand, just by calling the object `itemset` with the method `run`. This method returns:

1. `frequent_itemsets` dictionary where the keys indicates the frequent itemsets of size $k$.
2. `support_itemsets` containing the support of each of the frequent itemsets. 

The last variable will be used for calculating the association rules in the following section.

In [21]:
frequent_itemsets, support_itemsets = itemset.run(logging.DEBUG)

Generate catalog: 0.4192075729370117
Generate catalog: 0.4192075729370117
k=1: Candidates generation takes: 0.001055002212524414
k=1: Candidates generation takes: 0.001055002212524414
k=1: Prune takes: 0.0733642578125
k=1: Prune takes: 0.0733642578125
k=1: Ck=870 and Lk=375
k=1: Ck=870 and Lk=375
k=2: Candidates generation takes: 0.2848320007324219
k=2: Candidates generation takes: 0.2848320007324219
k=2: Prune takes: 16.243465423583984
k=2: Prune takes: 16.243465423583984
k=2: Ck=70125 and Lk=9
k=2: Ck=70125 and Lk=9
k=3: Candidates generation takes: 0.006543159484863281
k=3: Candidates generation takes: 0.006543159484863281
k=3: Prune takes: 0.002995014190673828
k=3: Prune takes: 0.002995014190673828
k=3: Ck=1 and Lk=1
k=3: Ck=1 and Lk=1
k=4: Candidates generation takes: 0.0005066394805908203
k=4: Candidates generation takes: 0.0005066394805908203
k=4: Prune takes: 1.430511474609375e-05
k=4: Prune takes: 1.430511474609375e-05
k=4: Ck=0 and Lk=0
k=4: Ck=0 and Lk=0
Total time: 17.06179

Finally, let us print some frequent itemsets of size $k=1,2 $ and $3$:

In [22]:
printify(frequent_itemsets[1], 10)

'{(349), (789), (653), (862), (490), (884), (151), (746), (579), (422)}'

In [23]:
printify(frequent_itemsets[2], 10)

'{(825, 704), (217, 346), (829, 368), (789, 829), (682, 368), (39, 825), (39, 704), (390, 227), (390, 722)}'

In [24]:
printify(frequent_itemsets[3], 10)

'{(825, 39, 704)}'

In [25]:
printify(frequent_itemsets[4], 10)

'}'

# Generating Association Rules

Once we have identified the frequent itemsets of a sales transaction given a support threshold $s$, we can generate association rules between them. We need an extra input parameter, the confidence threshold $c$. Given a rule $R: X \rightarrow Y$, the confidence of $R$ is the fraction of transactions containing $X \cup Y$ in all transaction that contain $X$. 

Good news, the confidence of a rule can be defined in terms of supports. For example, given the rule $R_1: \{a,b\} \rightarrow \{c,d\}$ the confidence of $R_1$ is calculated as:

$$\displaystyle c(R_1) = \frac{s(\{a,b,c,d\})}{s(\{a,b\})}$$

How do we generate new association rules? For each frequent itemset $I$, we generate a new rule with the form of $I - \{j\} \rightarrow j$ where $j$ is an element of the frequent itemset $I$. We repite this process for each generated rule until: a) the antecedent is empty b) the confidence of the rule is below the threshold level. 

Thanks to the confidence-based pruning property, if a rule $X \rightarrow Y$ does not satisfy the confidence threshold, then any rule $X' \rightarrow Y - X'$ where $X'$ is a subset of $X$, must not satisfy the confidence threshold as well.

# Real example

Let us take the largest $k$-frequent itemset from the last example and calculate the rules generated by it with a confindence of 0.5:

In [26]:
k_itemset = next(iter(l3))

association_rule05 = AssociationRule(0.5, support_itemsets)
rules_05 = association_rule05.generate_rules_from_single_itemset(k_itemset)

for rule in rules_05:
    print(rule)

{(39, 704)} -> {(825)}
{(704)} -> {(39, 825)}
{(39, 825)} -> {(704)}
{(704, 825)} -> {(39)}


We can observe how the number of rules retrieved by the algorithm changes when we increase/decrease the confidence threshold:

In [27]:
association_rule08 = AssociationRule(0.8, support_itemsets)
rules_08 = association_rule08.generate_rules_from_single_itemset(k_itemset)

for rule in rules_08:
    print(rule)

{(39, 704)} -> {(825)}
{(39, 825)} -> {(704)}
{(704, 825)} -> {(39)}


## All-in-one: Generating all association rules with confidence at least c

To conclude this notebook, we create all the rules generated by all the frequent itemsets calculated in the previous section by creating an `AssociationRule` object with the `frequent_itemsets` dictionary and calling the method `generate_all_rules`:

In [28]:
association_rule = AssociationRule(0.1, support_itemsets, frequent_itemsets)
rules = association_rule.generate_all_rules()
for rule in rules:
    print(rule)

{(825)} -> {(39)}
{(789)} -> {(829)}
{(825)} -> {(704)}
{(39)} -> {(704, 825)}
{(829)} -> {(789)}
{(390)} -> {(722)}
{(704)} -> {(39, 825)}
{(368)} -> {(682)}
{(390)} -> {(227)}
{(39, 704)} -> {(825)}
{(682)} -> {(368)}
{(227)} -> {(390)}
{(217)} -> {(346)}
{(704, 825)} -> {(39)}
{(825)} -> {(39, 704)}
{(39)} -> {(704)}
{(39, 825)} -> {(704)}
{(39)} -> {(825)}
{(346)} -> {(217)}
{(829)} -> {(368)}
{(704)} -> {(825)}
{(722)} -> {(390)}
{(704)} -> {(39)}
{(368)} -> {(829)}


# Appendix: Storing Support of Pairs

Lastly, we have decided to try a different approach for calculating the support of the itemsets. Since the candidate itemsets of size $2$ are the most numerous compared to any other $k$, mostly all the computation time will be spent checking the interesection of the two elements within each pair, as already explained (using `items_catalog` dictionary).  We have extended this dictionary and not only include the support of singletons but also the support of pairs. This is created just by simply creating combinations of pairs within each basket, and storing accordingly the list of baskets where that pair appears. 

The latter solution, of course, will spend more time creating the `items_catalog` but, on the other hand, the pruning step for $k=2$ will be immediately. Let us check the overall time of these two solutions, comparing the functions `create_items_catalog` and  `create_items_catalog_with_pairs`:

In [29]:
from apriori.utility import create_items_catalog_with_pairs

print('First solution:')
_, _ = itemset.run(logging.INFO, create_items_catalog)

print('Second solution:')
_, _ = itemset.run(logging.INFO, create_items_catalog_with_pairs)

First solution:


Total time: 18.213627815246582
Total time: 18.213627815246582


Second solution:


Total time: 10.310189485549927
Total time: 10.310189485549927


Comparing running times, we can agree that in spite of the second solution defeats the first proposal, this difference of time is at the expense of increasing the space complexity. However, given smaller values of the support threshold, the second solution becames much more efficient, since the number of candidates pairs considerable increases: 

In [30]:
itemset.support_threshold /= 10  # ten times smaller

print('First solution:')
_, _ = itemset.run(logging.INFO, create_items_catalog)

print('Second solution:')
_, _ = itemset.run(logging.INFO, create_items_catalog_with_pairs)

First solution:


Total time: 127.79799556732178
Total time: 127.79799556732178


Second solution:


Total time: 82.53843116760254
Total time: 82.53843116760254
